# HW2:  K-fold cross validation analysis Boston Housing dataset `506 samples and 13 feature variables`

---

**Citation**：
- [1] https://github.com/dmlc/xgboost.git
- [2] https://github.com/scikit-learn/scikit-learn.git
- [3] https://github.com/shap/shap
- [4] http://lib.stat.cmu.edu/datasets/boston

> ** 透過[1] XGBoost 與 [2] scikit-learn 套件實現**K-fold cross validation**。


In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import shap
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# 設定隨機種子確保可重現性
np.random.seed(42)

## 1. 載入 Boston Housing 資料集

In [16]:
data_path = 'data/boston.dataset.txt'
raw_df = pd.read_csv(data_path, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

column_names = [
    'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
    'PTRATIO', 'B', 'LSTAT'  # 移除 'MEDV'
]
df = pd.DataFrame(data, columns=column_names)
df['MEDV'] = target  # 單獨加入 target

print("資料載入成功！")
df.head()

資料載入成功！


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


## 2. 資料前處理

In [17]:
X = df.drop('MEDV', axis=1)
y = df['MEDV']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

In [ ]:
## 3. 定義評估函數

In [18]:
def evaluate_model(y_true, y_pred):
    mape = mean_absolute_percentage_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mape, rmse, r2

## 4. 5-Fold Cross Validation with XGBoost

In [19]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)
model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=300,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

mape_scores, rmse_scores, r2_scores = [], [], []
fold = 1

print("開始 5-Fold Cross Validation...\n")
print("-" * 60)
print(f"{'Fold':<6} {'MAPE':<10} {'RMSE':<10} {'R²':<10} {'Time(s)'}")
print("-" * 60)

for train_idx, val_idx in kf.split(X_scaled):
    X_train, X_val = X_scaled.iloc[train_idx], X_scaled.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_val)
    mape, rmse, r2 = evaluate_model(y_val, y_pred)
    
    mape_scores.append(mape)
    rmse_scores.append(rmse)
    r2_scores.append(r2)
    
    print(f"{fold:<6} {mape:<10.4f} {rmse:<10.4f} {r2:<10.4f}")
    fold += 1

print("-" * 60)
print(f"{'平均':<6} {np.mean(mape_scores):<10.4f} {np.mean(rmse_scores):<10.4f} {np.mean(r2_scores):<10.4f}")

開始 5-Fold Cross Validation...

------------------------------------------------------------
Fold   MAPE       RMSE       R²         Time(s)
------------------------------------------------------------
1      0.0984     2.4855     0.9158    
2      0.1000     2.8305     0.8980    
3      0.1161     3.4348     0.8703    
4      0.1028     3.0723     0.9096    
5      0.0937     2.8284     0.8887    
------------------------------------------------------------
平均     0.1022     2.9303     0.8965    
